In [8]:
import numpy as np
import pandas as pd

from dash import Dash, dcc, html
from dash.dependencies import Input, Output
import plotly.graph_objs as go

from tensorflow.keras.models import load_model
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score


In [10]:
# Code for APP

In [12]:
import numpy as np
import pandas as pd

from dash import Dash, dcc, html
from dash.dependencies import Input, Output
import plotly.graph_objs as go

from tensorflow.keras.models import load_model
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

# -----------------------------
# PATHS (your updated settings)
# -----------------------------
TATA_CSV_PATH = r"C:\Users\SUNYLoaner\Desktop\Code\ML Projects\SPS\NSE\NSE-DATA.csv"
STOCK_DATA_PATH = r"C:\Users\SUNYLoaner\Desktop\Code\ML Projects\SPS\stock_data\stock_data.csv"
MODEL_PATH = r"C:\Users\SUNYLoaner\Desktop\Code\ML Projects\SPS\src\saved_lstm_model.keras"

# -----------------------------
# DASH APP INIT
# -----------------------------
app = Dash(__name__)
server = app.server

# -----------------------------
# LOAD + PREPARE TATA DATA
# -----------------------------
df_nse = pd.read_csv(TATA_CSV_PATH)

# Ensure required columns exist
if "Date" not in df_nse.columns or "Close" not in df_nse.columns:
    raise ValueError("Your NSE-DATA.csv must contain 'Date' and 'Close' columns.")

df_nse["Date"] = pd.to_datetime(df_nse["Date"], errors="coerce")
df_nse["Close"] = pd.to_numeric(df_nse["Close"], errors="coerce")
df_nse = df_nse.dropna(subset=["Date", "Close"]).sort_values("Date").reset_index(drop=True)

# Keep only Date + Close
new_data = df_nse[["Date", "Close"]].copy()
new_data = new_data.set_index("Date")

# Convert to numpy for scaling
dataset = new_data[["Close"]].values  # shape (N, 1)

# Split train/valid (same as tutorial)
train_size = 987
if len(dataset) <= train_size + 60:
    raise ValueError("Dataset is too small. Need more rows than train_size + lookback.")

train = dataset[:train_size]
valid = dataset[train_size:]

# Scale (kept consistent with your tutorial style)
scaler = MinMaxScaler(feature_range=(0, 1))
scaled_data = scaler.fit_transform(dataset)

# Create training sequences (not needed for dashboard prediction, but kept for completeness)
lookback = 60
x_train, y_train = [], []
for i in range(lookback, train_size):
    x_train.append(scaled_data[i - lookback:i, 0])
    y_train.append(scaled_data[i, 0])

x_train = np.array(x_train).reshape(-1, lookback, 1)
y_train = np.array(y_train)

# -----------------------------
# LOAD MODEL (.keras) - compile=False removes optimizer warnings
# -----------------------------
model = load_model(MODEL_PATH, compile=False)

# -----------------------------
# BUILD INPUT WINDOWS FOR VALIDATION PREDICTION
# -----------------------------
valid_len = len(valid)

# Need last (valid_len + lookback) points so each validation day has 60 days history
inputs = dataset[len(dataset) - valid_len - lookback:]  # shape (valid_len+60, 1)
inputs_scaled = scaler.transform(inputs)

X_test = []
for i in range(lookback, inputs_scaled.shape[0]):
    X_test.append(inputs_scaled[i - lookback:i, 0])

X_test = np.array(X_test).reshape(-1, lookback, 1)

# Predict (scaled -> real)
pred_scaled = model.predict(X_test, verbose=0)
pred_close = scaler.inverse_transform(pred_scaled)  # shape (valid_len, 1)

# Build train/valid DataFrames for plotting
train_df = new_data.iloc[:train_size].copy()
valid_df = new_data.iloc[train_size:].copy()

# Add predictions
valid_df["Predictions"] = pred_close

# -----------------------------
# METRICS (on validation)
# -----------------------------
actual_close = valid_df[["Close"]].values  # shape (valid_len, 1)
pred_close = valid_df[["Predictions"]].values  # shape (valid_len, 1)

rmse = np.sqrt(mean_squared_error(actual_close, pred_close))
mae = mean_absolute_error(actual_close, pred_close)

# Avoid divide-by-zero in MAPE (rare but safe)
eps = 1e-9
mape = np.mean(np.abs((actual_close - pred_close) / (np.abs(actual_close) + eps))) * 100
accuracy_like = 100 - mape

r2 = r2_score(actual_close, pred_close)

metrics_card = html.Div(
    style={
        "maxWidth": "900px",
        "margin": "10px auto",
        "padding": "15px 20px",
        "border": "1px solid #ddd",
        "borderRadius": "12px",
        "backgroundColor": "#fafafa",
    },
    children=[
        html.H3("Model Performance (Validation)", style={"textAlign": "center", "marginBottom": "10px"}),
        html.Div(
            style={
                "display": "grid",
                "gridTemplateColumns": "repeat(3, 1fr)",
                "gap": "10px",
                "textAlign": "center",
            },
            children=[
                html.Div([html.B("RMSE (₹)"), html.Div(f"{rmse:.2f}")]),
                html.Div([html.B("MAE (₹)"), html.Div(f"{mae:.2f}")]),
                html.Div([html.B("MAPE (%)"), html.Div(f"{mape:.2f}")]),
                html.Div([html.B("Accuracy (~%)"), html.Div(f"{accuracy_like:.2f}")]),
                html.Div([html.B("R² Score"), html.Div(f"{r2:.3f}")]),
                html.Div([html.B("Lookback"), html.Div(f"{lookback} days")]),
            ],
        ),
        html.P(
            "Note: “Accuracy” here is reported as (100 − MAPE). This is a common portfolio metric for regression, not classification accuracy.",
            style={"fontSize": "12px", "color": "#555", "marginTop": "10px"},
        )
    ],
)

# -----------------------------
# LOAD MULTI-STOCK DATA
# -----------------------------
df_multi = pd.read_csv(STOCK_DATA_PATH)
required_cols = {"Date", "Stock", "High", "Low", "Volume"}
missing = required_cols - set(df_multi.columns)
if missing:
    raise ValueError(f"stock_data.csv is missing columns: {missing}")

df_multi["Date"] = pd.to_datetime(df_multi["Date"], errors="coerce")
df_multi["High"] = pd.to_numeric(df_multi["High"], errors="coerce")
df_multi["Low"] = pd.to_numeric(df_multi["Low"], errors="coerce")
df_multi["Volume"] = pd.to_numeric(df_multi["Volume"], errors="coerce")
df_multi = df_multi.dropna(subset=["Date", "Stock", "High", "Low", "Volume"])

# -----------------------------
# DASH LAYOUT
# -----------------------------
app.layout = html.Div([
    html.H1("Stock Price Analysis Dashboard", style={"textAlign": "center"}),

    dcc.Tabs(id="tabs", children=[

        # TAB 1: TATA Actual vs Predicted + Metrics
        dcc.Tab(label="TATA Stock Prediction (NSE-DATA.csv)", children=[
            html.Div([
                metrics_card,

                html.H2("Actual Closing Price (Train + Valid)", style={"textAlign": "center"}),

                dcc.Graph(
                    id="actual-data",
                    figure={
                        "data": [
                            go.Scatter(x=train_df.index, y=train_df["Close"], mode="lines", name="Train Close"),
                            go.Scatter(x=valid_df.index, y=valid_df["Close"], mode="lines", name="Valid Close"),
                        ],
                        "layout": go.Layout(
                            title="Actual Close Price",
                            xaxis={"title": "Date"},
                            yaxis={"title": "Closing Price"},
                            height=500
                        )
                    }
                ),

                html.H2("LSTM Predicted vs Actual (Validation)", style={"textAlign": "center"}),

                dcc.Graph(
                    id="predicted-data",
                    figure={
                        "data": [
                            go.Scatter(x=valid_df.index, y=valid_df["Close"], mode="lines", name="Actual Valid Close"),
                            go.Scatter(x=valid_df.index, y=valid_df["Predictions"], mode="lines", name="Predicted Close"),
                        ],
                        "layout": go.Layout(
                            title="Predicted vs Actual (Validation)",
                            xaxis={"title": "Date"},
                            yaxis={"title": "Closing Price"},
                            height=500
                        )
                    }
                ),
            ])
        ]),

        # TAB 2: Multi-stock charts (visualization only)
        dcc.Tab(label="Multi-Stock Visuals (stock_data.csv)", children=[
            html.Div([
                html.H2("High vs Low Prices", style={"textAlign": "center"}),

                dcc.Dropdown(
                    id="my-dropdown",
                    options=[
                        {"label": "Tesla", "value": "TSLA"},
                        {"label": "Apple", "value": "AAPL"},
                        {"label": "Facebook", "value": "FB"},
                        {"label": "Microsoft", "value": "MSFT"},
                    ],
                    multi=True,
                    value=["FB"],
                    style={"display": "block", "marginLeft": "auto", "marginRight": "auto", "width": "60%"}
                ),

                dcc.Graph(id="highlow"),

                html.H2("Market Volume", style={"textAlign": "center"}),

                dcc.Dropdown(
                    id="my-dropdown2",
                    options=[
                        {"label": "Tesla", "value": "TSLA"},
                        {"label": "Apple", "value": "AAPL"},
                        {"label": "Facebook", "value": "FB"},
                        {"label": "Microsoft", "value": "MSFT"},
                    ],
                    multi=True,
                    value=["FB"],
                    style={"display": "block", "marginLeft": "auto", "marginRight": "auto", "width": "60%"}
                ),

                dcc.Graph(id="volume")
            ], className="container")
        ])
    ])
])

# -----------------------------
# CALLBACKS
# -----------------------------
@app.callback(Output("highlow", "figure"), [Input("my-dropdown", "value")])
def update_highlow(selected_dropdown):
    dropdown = {"TSLA": "Tesla", "AAPL": "Apple", "FB": "Facebook", "MSFT": "Microsoft"}

    traces = []
    for stock in selected_dropdown:
        df_s = df_multi[df_multi["Stock"] == stock]
        traces.append(go.Scatter(
            x=df_s["Date"], y=df_s["High"],
            mode="lines", opacity=0.7,
            name=f"High {dropdown.get(stock, stock)}"
        ))
        traces.append(go.Scatter(
            x=df_s["Date"], y=df_s["Low"],
            mode="lines", opacity=0.6,
            name=f"Low {dropdown.get(stock, stock)}"
        ))

    figure = {
        "data": traces,
        "layout": go.Layout(
            height=600,
            title=f"High & Low Prices for {', '.join(dropdown.get(i, i) for i in selected_dropdown)} Over Time",
            xaxis={
                "title": "Date",
                "rangeselector": {
                    "buttons": [
                        {"count": 1, "label": "1M", "step": "month", "stepmode": "backward"},
                        {"count": 6, "label": "6M", "step": "month", "stepmode": "backward"},
                        {"step": "all"}
                    ]
                },
                "rangeslider": {"visible": True},
                "type": "date"
            },
            yaxis={"title": "Price (USD)"}
        )
    }
    return figure


@app.callback(Output("volume", "figure"), [Input("my-dropdown2", "value")])
def update_volume(selected_dropdown_value):
    dropdown = {"TSLA": "Tesla", "AAPL": "Apple", "FB": "Facebook", "MSFT": "Microsoft"}

    traces = []
    for stock in selected_dropdown_value:
        df_s = df_multi[df_multi["Stock"] == stock]
        traces.append(go.Scatter(
            x=df_s["Date"], y=df_s["Volume"],
            mode="lines", opacity=0.7,
            name=f"Volume {dropdown.get(stock, stock)}"
        ))

    figure = {
        "data": traces,
        "layout": go.Layout(
            height=600,
            title=f"Market Volume for {', '.join(dropdown.get(i, i) for i in selected_dropdown_value)} Over Time",
            xaxis={
                "title": "Date",
                "rangeselector": {
                    "buttons": [
                        {"count": 1, "label": "1M", "step": "month", "stepmode": "backward"},
                        {"count": 6, "label": "6M", "step": "month", "stepmode": "backward"},
                        {"step": "all"}
                    ]
                },
                "rangeslider": {"visible": True},
                "type": "date"
            },
            yaxis={"title": "Volume"}
        )
    }
    return figure


if __name__ == "__main__":
    # Newer Dash versions use app.run (not run_server)
    # use_reloader=False is safer if you run inside Jupyter notebooks
    app.run(debug=True, use_reloader=False)
